In [33]:
with open("./input.txt", "r",encoding="utf-8") as file:
    data = file.read()
    # print(data)
import torch

voc=sorted(list(set(data)))
# print(len(data))
# print(list(set(data)))
# print(set(data))
# print(voc)
chtoi={c:i for i,c in enumerate(voc)}
itoch={i:c for i,c in enumerate(voc)}
encoding=lambda s: [chtoi[c] for c in s]
decoding=lambda s: "".join([itoch[c] for c in s])


# print(encoding("hello world"))
# print(decoding(encoding("hello world")))

datatset=torch.tensor(encoding(data),dtype=torch.long)
# print(datatset[:100])
n=int(0.9*len(datatset))
trainset=datatset[:n]
testset=datatset[n:]
block_size=8
x_train=datatset[:block_size+1]
y_train=datatset[1:block_size+1]
# print(x_train)
for i in range(block_size):
    context=x_train[:i+1]
    target=y_train[i]
    # print(f"context: {context}, target: {target}")
torch.manual_seed(42)
batch_size=4
block_size=8
def getbatch(split):
    data=trainset if split=="train" else testset
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack(([data[i:i+block_size] for i in ix]))
    y=torch.stack(([data[i+1:i+block_size+1] for i in ix]))
    # print(ix)
    return x,y
xb,yb=getbatch("train")
print(xb)
print(yb)

for b in range(batch_size):
    for a in range(block_size):
        context=xb[b,:a+1]
        target=yb[b,a]
        print(f"context: {context}, target: {target}")
    


tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
context: tensor([57]), target: 1
context: tensor([57,  1]), target: 46
context: tensor([57,  1, 46]), target: 47
context: tensor([57,  1, 46, 47]), target: 57
context: tensor([57,  1, 46, 47, 57]), target: 1
context: tensor([57,  1, 46, 47, 57,  1]), target: 50
context: tensor([57,  1, 46, 47, 57,  1, 50]), target: 53
context: tensor([57,  1, 46, 47, 57,  1, 50, 53]), target: 60
context: tensor([1]), target: 58
context: tensor([ 1, 58]), target: 46
context: tensor([ 1, 58, 46]), target: 43
context: tensor([ 1, 58, 46, 43]), target: 56
context: tensor([ 1, 58, 46, 43, 56]), target: 43
context: tensor([ 1, 58, 46, 43, 56, 43]), target: 1
context: tensor([ 

In [34]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self, xb,yb=None):
        logits=self.token_embedding_table(xb)
      
        if yb is not None:
            B,C,T=logits.shape
            logits=logits.view(B*C,T)
            yb=yb.view(B*C)

            loss=F.cross_entropy(logits,yb)
        else:
            loss=None
        return logits,loss

    def generate(self, idx, max_new_tokens): 
        # idx is (B, T) array of indices in the current context 
        for _ in range(max_new_tokens):  
            # get the predictions 
            logits, loss = self(idx)  

            # focus only on the last time step 
            # print(logits)
            logits = logits[:, -1, :]  # becomes (B, C)  
            # print(logits)

            # apply softmax to get probabilities  
            probs = F.softmax(logits, dim=-1)  # (B, C)  

            # sample from the distribution  
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)  

            # append sampled index to the running sequence  
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1) 
            # print(idx) 

        return idx

m=BigramLanguageModel(len(voc))
logit,loss=m(xb,yb)
# print(logit.shape)
# print(loss)
        
# print(len(voc))

print(decoding(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


uoiaF$z
M?kI;h
DbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS
g3lEb&ZQ,l;:m;lpcNN
KpVEYRIIM,'hCRbMAcWTkrnH


In [40]:
m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)

tensor([[ 0, 15, 39, 52, 46, 40, 39, 44, 64, 48, 35, 56,  6, 25, 60, 47, 39,  1,
         58,  5, 28, 43, 50,  5, 42,  1,  1, 43,  1, 63, 46, 39,  9, 52,  0, 58,
         43, 17, 61, 44, 18, 24, 17, 21, 25, 13, 25, 28, 20, 20, 17, 31, 49, 12,
          7, 40, 53, 53, 59, 51, 39, 51, 32, 47, 39,  7, 21, 52, 43, 51, 39, 42,
          2, 41,  8,  0, 24, 32,  1, 17, 35, 20, 24, 14,  2,  4,  6,  1, 58, 59,
         25, 60, 49, 12, 61, 33,  1, 57, 57, 47, 53]])

In [36]:
len("""gBXCYUEBUkTrhlwhYvCHU3jVQWIOVFHSbBtlnSfFoHZK.nHR&AcjvfzzJ.K,HCYUui-xd
L;Wu3VnSgv.$afs
BqpdHVLgwcOwN """)

100

In [37]:
optimizer=torch.optim.AdamW(m.parameters(),lr=0.001)


In [41]:
batch_size = 32

for steps in range(10000):
    xb, yb = getbatch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if steps % 100 == 0:
        print(f"Step {steps+1}, Loss: {loss.item():.4f}")


Step 1, Loss: 3.1633
Step 101, Loss: 3.0629
Step 201, Loss: 3.1231
Step 301, Loss: 2.9733
Step 401, Loss: 3.0888
Step 501, Loss: 2.9932
Step 601, Loss: 2.8694
Step 701, Loss: 2.9049
Step 801, Loss: 2.9283
Step 901, Loss: 2.8630
Step 1001, Loss: 2.8598
Step 1101, Loss: 2.9170
Step 1201, Loss: 2.7032
Step 1301, Loss: 2.7869
Step 1401, Loss: 2.7989
Step 1501, Loss: 2.6292
Step 1601, Loss: 2.7759
Step 1701, Loss: 2.7046
Step 1801, Loss: 2.6092
Step 1901, Loss: 2.5815
Step 2001, Loss: 2.6210
Step 2101, Loss: 2.6367
Step 2201, Loss: 2.6492
Step 2301, Loss: 2.6701
Step 2401, Loss: 2.6539
Step 2501, Loss: 2.6140
Step 2601, Loss: 2.5384
Step 2701, Loss: 2.7407
Step 2801, Loss: 2.5222
Step 2901, Loss: 2.6269
Step 3001, Loss: 2.5507
Step 3101, Loss: 2.5608
Step 3201, Loss: 2.5470
Step 3301, Loss: 2.5149
Step 3401, Loss: 2.5263
Step 3501, Loss: 2.4625
Step 3601, Loss: 2.5614
Step 3701, Loss: 2.6202
Step 3801, Loss: 2.5995
Step 3901, Loss: 2.5212
Step 4001, Loss: 2.5573
Step 4101, Loss: 2.5108
Step

In [ ]:
print(decoding(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))